In [39]:
# import dependencies
import pandas as pd

In [8]:
#import file
purchase_data = pd.read_json('purchase_data.json')

purchase_data.to_csv('test.csv')

In [9]:
#Define convert_to_currency for use in mapping
def convert_to_currency(number):
    return '${:,.2f}'.format(number)

In [10]:
#overview data
purchase_data.info()
purchase_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780 entries, 0 to 779
Data columns (total 6 columns):
Age          780 non-null int64
Gender       780 non-null object
Item ID      780 non-null int64
Item Name    780 non-null object
Price        780 non-null float64
SN           780 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 42.7+ KB


,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [11]:
#player count

user_count = purchase_data['SN'].nunique()

print('There are ' + str(user_count) + ' unique players.')

There are 573 unique players.


In [21]:
### Purchasing Analysis (Total)

#Number of Unique Items
item_count = purchase_data['Item ID'].nunique()

#Average Purchase Price
#Total Number of Purchases
#Total Revenue

total_revenue = purchase_data['Price'].sum()

purchase_count = len(purchase_data)

average_purchase = total_revenue / purchase_count

purchase_dict = [{'Number of Unique Items': item_count,
                       'Average Price': average_purchase,
                       'Number of Purchases': purchase_count,
                       'Total Revenue': total_revenue}]

purchase = pd.DataFrame(data=purchase_dict)

#Re-ordering the column, for some reason the function above puts them in arbitrary order
purchase = purchase[['Number of Unique Items', 
                                                 'Average Price', 
                                                 'Number of Purchases', 
                                                 'Total Revenue']]

purchase['Average Price'] = purchase['Average Price'].map(convert_to_currency)
purchase['Total Revenue'] = purchase['Total Revenue'].map(convert_to_currency)

purchase

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [22]:
### Gender Demographics
   #Percentage and Count of Male Players
   #Percentage and Count of Female Players
   #Percentage and Count of Other / Non-Disclosed

gender_data = purchase_data.set_index('Gender')

total_count = len(gender_data)
male_count = len(gender_data.loc['Male'])
female_count = len(gender_data.loc['Female'])
other_count = len(gender_data.loc['Other / Non-Disclosed'])

gender_dict = {'Gender' : ['Male', 'Female', 'Other / Nondisclosed'],
               'Percentage of Players': [male_count/total_count, female_count/total_count, other_count/total_count],
               'Player Count': [male_count, female_count, other_count]
              }

gender = pd.DataFrame(gender_dict)

gender['Percentage of Players'] = round(gender['Percentage of Players']*100, 2)

gender = gender.set_index('Gender')

gender


,Percentage of Players,Player Count
Gender,,
Male,81.15,633
Female,17.44,136
Other / Nondisclosed,1.41,11


In [37]:
### Age Demographics
   #Percentage and Count of All Age Bins
    
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]

bin_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchase_data['Age Bracket'] = pd.cut(purchase_data['Age'], bins, labels=bin_names)


agebin = purchase_data.set_index('Age Bracket')


total_count = len(agebin)
count_less10 = len(agebin.loc['<10'])
count_10to14 = len(agebin.loc['10-14'])
count_15to19 = len(agebin.loc['15-19'])
count_20to24 = len(agebin.loc['20-24'])
count_25to29 = len(agebin.loc['25-29'])
count_30to34 = len(agebin.loc['30-34'])
count_35to39 = len(agebin.loc['35-39'])
count_40plus = len(agebin.loc['40+'])


agebin_dict = {'Age Brackets' : ['<10', 
                                 '10-14', 
                                 '15-19', 
                                 '20-24', 
                                 '25-29', 
                                 '30-34', 
                                 '35-39', 
                                 '40+'],
               'Percentage of Players': [count_less10 / total_count,
                                        count_10to14 / total_count,
                                        count_15to19 / total_count,
                                        count_20to24 / total_count,
                                        count_25to29 / total_count,
                                        count_30to34 / total_count,
                                        count_35to39 / total_count,
                                        count_40plus / total_count],
               'Player Count': [count_less10,
                                count_10to14,
                                count_15to19,
                                count_20to24,
                                count_25to29,
                                count_30to34,
                                count_35to39,
                                count_40plus]
              }

age = pd.DataFrame(agebin_dict)

age['Percentage of Players'] = round(age['Percentage of Players'] * 100, 2)

age



,Age Brackets,Percentage of Players,Player Count
0,<10,3.59,28
1,10-14,4.49,35
2,15-19,17.05,133
3,20-24,43.08,336
4,25-29,16.03,125
5,30-34,8.21,64
6,35-39,5.38,42
7,40+,2.18,17


In [29]:
### Purchasing Analysis (Gender)

#The below each broken by gender
  # Purchase Count
  # Average Purchase Price
  # Total Purchase Value
  # Normalized Totals

group_gender = purchase_data.groupby('Gender')

gender_purchase = group_gender.agg({'Item ID': 'count',
                          'Price': ['mean', 'sum'],
                          'SN': pd.Series.nunique
                          })

#rename columns, flattened from multi-index to single index
gender_purchase.columns = ['Purchase Count', 
                           'Average Price', 
                           'Total Purchase Value', 
                           'Unique User Count']

gender_purchase['Normalized Per User Price'] = gender_purchase['Total Purchase Value'] / gender_purchase['Unique User Count']

#remove 'Unique User Count', only used for calculation above
del gender_purchase['Unique User Count']

gender_purchase['Average Price'] = gender_purchase['Average Price'].map(convert_to_currency)
gender_purchase['Total Purchase Value'] = gender_purchase['Total Purchase Value'].map(convert_to_currency)
gender_purchase['Normalized Per User Price'] = gender_purchase['Normalized Per User Price'].map(convert_to_currency)


gender_purchase

,Purchase Count,Average Price,Total Purchase Value,Normalized Per User Price
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [30]:
### Purchasing Analysis (Age)

# The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
  # Purchase Count
  # Average Purchase Price
  # Total Purchase Value
  # Normalized Totals

group_agebin = purchase_data.groupby('Age Bracket')

age_purchase = group_agebin.agg({'Item ID': 'count',
                        'Price': ['mean', 'sum'],
                        'SN': pd.Series.nunique
                        })

age_purchase.columns = ['Purchase Count', 
                'Average Price', 
                'Total Purchase Value', 
                'Unique User Count']

age_purchase['Normalized Per User Price'] = age_purchase['Total Purchase Value'] / age_purchase['Unique User Count']

del age_purchase['Unique User Count']

age_purchase['Average Price'] = age_purchase['Average Price'].map(convert_to_currency)
age_purchase['Total Purchase Value'] = age_purchase['Total Purchase Value'].map(convert_to_currency)
age_purchase['Normalized Per User Price'] = age_purchase['Normalized Per User Price'].map(convert_to_currency)

age_purchase

,Purchase Count,Average Price,Total Purchase Value,Normalized Per User Price
Age Bracket,,,,
<10,28,$2.98,$83.46,$4.39
10-14,35,$2.77,$96.95,$4.22
15-19,133,$2.91,$386.42,$3.86
20-24,336,$2.91,$978.77,$3.78
25-29,125,$2.96,$370.33,$4.26
30-34,64,$3.08,$197.25,$4.20
35-39,42,$2.84,$119.40,$4.42
40+,17,$3.16,$53.75,$4.89


In [95]:
### Top Spenders

# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
  # SN
  # Purchase Count
  # Average Purchase Price
  # Total Purchase Value

group_sn = purchase_data.groupby('SN')

#aggregate relevant data
spenders = group_sn.agg({'Item ID': 'count',
             'Price': ['mean', 'sum']})

#rename columns
spenders.columns = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value']

#isolate top 5 spenders
top_5_spenders = spenders.nlargest(5, 'Total Purchase Value')

top_5_spenders['Average Purchase Price'] = top_5_spenders['Average Purchase Price'].map(convert_to_currency)
top_5_spenders['Total Purchase Value'] = top_5_spenders['Total Purchase Value'].map(convert_to_currency)


top_5_spenders


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [26]:
### Most Popular Items

# Identify the 5 most popular items by purchase count, then list (in a table):
  # Item ID
  # Item Name
  # Purchase Count
  # Item Price
  # Total Purchase Value

group_item = purchase_data.groupby(['Item ID', 'Item Name'])

items = group_item.agg({'Item ID': 'count',
             'Price': ['mean', 'sum']})

items.columns = ['Purchase Count', 'Item Price', 'Total Purchase Price']
items = items.sort_values(by=['Purchase Count', 'Total Purchase Price'], ascending=False)

top_5_items = items.nlargest(5, 'Purchase Count')

top_5_items['Item Price']= top_5_items['Item Price'].map(convert_to_currency)
top_5_items['Total Purchase Price'] = top_5_items['Total Purchase Price'].map(convert_to_currency)

top_5_items

,,Purchase Count,Item Price,Total Purchase Price
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
34,Retribution Axe,9,$4.14,$37.26
31,Trickster,9,$2.07,$18.63
13,Serenity,9,$1.49,$13.41


In [27]:
### Most Profitable Items

# Identify the 5 most profitable items by total purchase value, then list (in a table):
  # Item ID
  # Item Name
  # Purchase Count
  # Item Price
  # Total Purchase Value

top_5_items_profit = items.nlargest(5, 'Total Purchase Price')

top_5_items_profit['Item Price']= top_5_items_profit['Item Price'].map(convert_to_currency)
top_5_items_profit['Total Purchase Price'] = top_5_items_profit['Total Purchase Price'].map(convert_to_currency)

top_5_items_profit

,,Purchase Count,Item Price,Total Purchase Price
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
